<a href="https://colab.research.google.com/github/vishnuraghavendar2019/Exploratory-data-analysis-DA-21BDS0145/blob/main/EDA%20%20DA%2021BDS0145.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#reading csv file
import numpy as np
import pandas as pd


url = 'https://raw.githubusercontent.com/salemprakash/EDA/main/Data/constituents.csv'
data = pd.read_csv(url)
data.head(10)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
5,ADBE,Adobe Inc.,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982
6,AMD,Advanced Micro Devices,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488,1969
7,AES,AES Corporation,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",1998-10-02,874761,1981
8,AFL,Aflac,Financials,Life & Health Insurance,"Columbus, Georgia",1999-05-28,4977,1955
9,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,"Santa Clara, California",2000-06-05,1090872,1999


In [35]:
#column names
data.columns

Index(['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')

In [36]:
#datatypes
data.dtypes

,0
Symbol,object
Security,object
GICS Sector,object
GICS Sub-Industry,object
Headquarters Location,object
Date added,object
CIK,int64
Founded,object


In [37]:
#rows and column
data.shape

(503, 8)

In [38]:
#mean of CIK
data_mean = np.mean(data["CIK"])
data_mean

821753.648111332

In [39]:
data.describe()

,CIK
count,5.030000e+02
mean,8.217536e+05
std,5.665200e+05
min,1.800000e+03
25%,1.005050e+05
50%,9.000750e+05
75%,1.175188e+06
max,2.005951e+06


**Data handling - checking for NaN values**

In [40]:
#check null values
data.isnull()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
498,False,False,False,False,False,False,False,False
499,False,False,False,False,False,False,False,False
500,False,False,False,False,False,False,False,False
501,False,False,False,False,False,False,False,False


the given csv file have no null values

In [41]:
#checking the number of null values in each attribute
data.isnull().sum()

,0
Symbol,0
Security,0
GICS Sector,0
GICS Sub-Industry,0
Headquarters Location,0
Date added,0
CIK,0
Founded,0


In [42]:
#checking total null values
data.isnull().sum().sum()

0

In [43]:
#Descriptive statistics
mean = data["CIK"].mean()
median =data["CIK"].median()
mode = data["CIK"].mode()
print(mean , median, mode)

821753.648111332 900075.0 0    1564708
1    1652044
2    1754301
Name: CIK, dtype: int64
